### Reconstructions of the trained models
- splitted data: 20, 40 ,60 ,80 ,100
- FC_training:
- video_output:


### Splitted_data
This is a regular model

In [1]:
import mxnet as mx
import numpy as np

from mxnet import Context, cpu, gpu
from tqdm import tqdm
 
from generator_vgg import Generator

from util import Device
import matplotlib.pyplot as plt 

from mxnet.ndarray import concat
import modules
from generator_vgg import Generator_prediction
import modules_for_loading

In [2]:
def save_construct(runname, epoch,  ):
    rel_dir = 'saved_models'
    set_t = 'testing'
    time_range=5
    in_chan = 15
    
    device = Device.GPU2
    context = cpu() if device.value == -1 else gpu(device.value)

    iterator = modules.make_iterator_preprocessed(set_t, 'V1', 'V2', 'V3', shift=4, batch_size=16, time_range=time_range)

    outputs_lst = []
    targets_lst = []
    inputs_lst = []
    with Context(context):
        RFlocs_V1_overlapped_avg = modules.get_RFs('V1', context)
        RFlocs_V2_overlapped_avg = modules.get_RFs('V2', context)
        RFlocs_V3_overlapped_avg = modules.get_RFs('V3', context)
        
        generator = Generator_prediction(in_chan, context)
        generator.network.load_parameters(f'{rel_dir}/{runname}/netG_{epoch}.model', ctx=context)

    for RFsignalsV1, RFsignalsV2, RFsignalsV3, targets in tqdm(iterator):
        with Context(context):


            # -----
            # Inputs
            # -----
            inputsV1 = modules.get_inputsROI(RFsignalsV1, RFlocs_V1_overlapped_avg, context)
            inputsV2 = modules.get_inputsROI(RFsignalsV2, RFlocs_V2_overlapped_avg, context)
            inputsV3 = modules.get_inputsROI(RFsignalsV3, RFlocs_V3_overlapped_avg, context)

            inputs = concat(inputsV1, inputsV2, inputsV3, dim=1)

            outputs = generator.network(inputs)
            targets = targets.as_in_context(context).transpose((0,1,2,3))

        outputs_np = outputs.asnumpy()
        targets_np = targets.asnumpy()
        inputs_np = inputs.asnumpy()
        
#         outputs_np = np.clip(255 * outputs, 0, 255).
#         targets_np = np.clip(255 * targets, 0, 255)

        outputs_lst.append(outputs_np)
        targets_lst.append(targets_np)
        inputs_lst.append(inputs_np)

    outputs = np.concatenate(outputs_lst)
    targets = np.concatenate(targets_lst)
    inputs = np.concatenate(inputs_lst)
    
    np.savez_compressed(f'../final_reconstructions/{runname}', inputs = inputs, targets = targets, outputs = outputs)

In [3]:
save_construct('splitted_data_0.2', 49)
save_construct('splitted_data_0.4', 49)
save_construct('splitted_data_0.6', 49)
save_construct('splitted_data_0.8', 49)

100%|██████████| 65/65 [00:05<00:00, 10.16it/s]


In [4]:
save_construct('splitted_data_1.0', 50)

100%|██████████| 65/65 [00:05<00:00,  9.98it/s]
